<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_Scripts/XRD_File_Corrections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XRD File Corrections

This script helps make various corrections to XRD files in an xy format. All files must be located in an accessible google drive directory.

Current Corrections Available:

*   Background Subtraction




## Imports

In [20]:
from google.colab import drive  #allows downloading and accessing files on drive
from google.colab import files  #allows downloading and accessing files on drive
import shutil                   #allows uploading files to drive
import glob                     #useful for collecting and organizing files
import csv
import numpy as np              #math library
import pandas as pd             #data organization library
import scipy as sp              #math library
import re                       #string processing library
from scipy import optimize      #math fitting library
import matplotlib.pylab as plt  #allows for plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


## Extract Data from Google Drive

In [2]:
# log into google drive with your account
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [67]:
# Directory Locations for information
XRD_files_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/'
XRD_tmps_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/scalar_data/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary.csv'
XRD_subtraction_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Quartz_Capillary_no_FG_PDF_1sec/integration/'
XRD_subtraction_tmps_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Quartz_Capillary_no_FG_PDF_1sec/scalar_data/Quartz_Capillary_no_FG_PDF_1sec_ALL_INFO.csv'

In [68]:
# Read in XRD file temperature data
data1 = pd.read_csv(XRD_tmps_path)
data1 = data1.astype(float)
data1 = data1.sort_values(by='time')
# Convert eurotherm (i.e. control) temperature into sample temperature using a fit
C0,C1,C2,C3 = 4.91, 0.8519, 0.0005649, -2.536*10**(-7)
data1['sample_temp'] = round(C3*data1['eurotherm_temperature']**3 + C2*data1['eurotherm_temperature']**2 + C1*data1['eurotherm_temperature']**1 + C0,1)

# Read in XRD subtraction file temperature data
data2  = pd.read_csv(XRD_subtraction_tmps_path)
data2 = data2.astype(float)
data2 = data2.sort_values(by='time')
# Convert eurotherm (i.e. control) temperature into sample temperature using a fit
C0,C1,C2,C3 = 4.91, 0.8519, 0.0005649, -2.536*10**(-7)
data2['sample_temp'] = round(C3*data2['eurotherm_temperature']**3 + C2*data2['eurotherm_temperature']**2 + C1*data2['eurotherm_temperature']**1 + C0,1)

data1
data2

,time,pe1_stats1_total,eurotherm_temperature,eurotherm_power,chi_max,chi_argmax,gr_max,gr_argmax,seq_num,real_seq_num,sample_temp
0,1.718247e+09,2.106761e+10,30.0,6.1,1.484337,618.14580,0.0,0.0,1.0,1.0,31.0
1,1.718247e+09,2.106841e+10,30.0,7.6,1.484337,621.98883,0.0,0.0,2.0,2.0,31.0
2,1.718247e+09,2.106894e+10,30.0,9.0,1.503549,623.33905,0.0,0.0,3.0,3.0,31.0
3,1.718247e+09,2.106952e+10,31.0,10.7,1.484337,625.36830,0.0,0.0,4.0,4.0,31.9
4,1.718247e+09,2.106991e+10,31.0,11.7,1.484337,627.42755,0.0,0.0,5.0,5.0,31.9
...,...,...,...,...,...,...,...,...,...,...,...
463,1.718248e+09,2.107230e+10,464.0,44.8,1.484337,631.12310,0.0,0.0,113.0,464.0,496.5
464,1.718248e+09,2.107229e+10,463.0,47.2,1.484337,629.37960,0.0,0.0,114.0,465.0,495.3
465,1.718248e+09,2.107214e+10,463.0,46.7,1.484337,631.71783,0.0,0.0,115.0,466.0,495.3
466,1.718248e+09,2.107180e+10,464.0,46.7,1.484337,628.12430,0.0,0.0,116.0,467.0,496.5


In [70]:
XRD_files = glob.glob(XRD_files_path + '*275e1a*.xy')
XRD_sub_files = glob.glob(XRD_subtraction_path + '*225655*.xy')
print(len(XRD_files), len(XRD_sub_files))

950 117


In [63]:
for file in XRD_files[:30]:
  #new_name = file.replace(".xy","_quartz_subtraction.xy")
  #data = pd.read_csv(file, skiprows=1, header=None, sep="\s+").astype(float)
  file_number = float(re.findall(r'\d+',file)[-1])                                      # file number which is related to the amount of time after which the experiment started
  XRD_temp = data1[data1['seq_num']==file_number]['sample_temp'].item()  # eurotherm temperature for the associated XRD file number
  XRD_sub_file_number = data2[data2['sample_temp']==XRD_temp]
  if XRD_sub_file_number.empty:
    continue
  else:
    XRD_sub_file_number = XRD_sub_file_number.iloc[0]['seq_num'].item()
    print(XRD_temp, XRD_sub_file_number)
  #data.to_csv(new_name, header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index

161.8 19.0
160.7 18.0
160.7 18.0
160.7 18.0
160.7 18.0
159.7 17.0
159.7 17.0
160.7 18.0
160.7 18.0
159.7 17.0
159.7 17.0
159.7 17.0
160.7 18.0
160.7 18.0
148.6 5.0
41.7 12.0
124.6 98.0
161.8 19.0
133.6 109.0
140.5 116.0
160.7 18.0
42.6 13.0
148.6 5.0
160.7 18.0
158.7 16.0
